# Defining a Fixed-Floating Interest Rate Swap

Simple introduction to the inputs required to define a Libor deposit and a FRA

In [1]:
from financepy.finutils import *
from financepy.products.libor import *

####################################################################
# FINANCEPY BETA Version 0.180 - This build:  22 Sep 2020 at 22:40 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#    For bug reports and comments - allmypythonprojects@gmail.com  #
####################################################################



## Creating the Swap

We begin by setting the market conventions we wish to use.

In [2]:
swapCalendarType = FinCalendarTypes.TARGET
busDayAdjustType = FinBusDayAdjustTypes.FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

We then define the Fixed Leg

In [3]:
fixedCoupon = 0.05
fixedFreqType = FinFrequencyTypes.ANNUAL
fixedDayCountType = FinDayCountTypes.ACT_360

And then we define the Floating Leg

In [4]:
floatSpread = 0.0
floatFreqType = FinFrequencyTypes.SEMI_ANNUAL
floatDayCountType = FinDayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [5]:
swapType = FinLiborSwapTypes.RECEIVER
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

In [6]:
startDate = FinDate(20, 6, 2018)
maturityDate = FinDate(20, 9, 2025)

We can now create the swap object.

In [7]:
swap = FinLiborSwap(startDate,
                    maturityDate,
                    swapType,
                    fixedCoupon,
                    fixedFreqType,
                    fixedDayCountType,
                    notional,
                    floatSpread,
                    floatFreqType,
                    floatDayCountType,
                    swapCalendarType,
                    busDayAdjustType,
                    dateGenRuleType)

In [8]:
print(swap)

OBJECT TYPE: FinLiborSwap
START DATE: WED 20 JUN 2018
TERMINATION DATE: SAT 20 SEP 2025
MATURITY DATE: MON 22 SEP 2025
NOTIONAL: 10000000
SWAP TYPE: FinLiborSwapTypes.RECEIVER
FIXED COUPON: 0.05
FLOAT SPREAD: 0.0
FIXED FREQUENCY: FinFrequencyTypes.ANNUAL
FLOAT FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.ACT_360
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.TARGET
BUS DAY ADJUST: FinBusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: FinDateGenRuleTypes.BACKWARD



In [10]:
swap.fixedDates()

[THU 20 SEP 2018,
 FRI 20 SEP 2019,
 MON 21 SEP 2020,
 MON 20 SEP 2021,
 TUE 20 SEP 2022,
 WED 20 SEP 2023,
 FRI 20 SEP 2024,
 MON 22 SEP 2025]

In [11]:
swap.floatDates()

[THU 20 SEP 2018,
 WED 20 MAR 2019,
 FRI 20 SEP 2019,
 FRI 20 MAR 2020,
 MON 21 SEP 2020,
 MON 22 MAR 2021,
 MON 20 SEP 2021,
 MON 21 MAR 2022,
 TUE 20 SEP 2022,
 MON 20 MAR 2023,
 WED 20 SEP 2023,
 WED 20 MAR 2024,
 FRI 20 SEP 2024,
 THU 20 MAR 2025,
 MON 22 SEP 2025]

## Valuation

In [12]:
valuationDate = FinDate(20,3,2020)

In [13]:
settlementDate = valuationDate.addWorkDays(0)

This is just using a flat curve without any Libor accrual factors.

In [15]:
from financepy.market.curves.FinDiscountCurveFlat import FinDiscountCurveFlat

In [17]:
discountCurve = FinDiscountCurveFlat(valuationDate, 0.05, FinFrequencyTypes.SEMI_ANNUAL)

Need to specify the next fixing on the floating leg

In [18]:
swap.value(settlementDate, discountCurve, discountCurve)

254993.6463796678

We can examine the individual legs - here is the floating leg valuation

In [21]:
swap.printFloatLegPV()

START DATE: WED 20 JUN 2018
MATURITY DATE: MON 22 SEP 2025
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
         *** FIRST FLOATING RATE PAYMENT IS IMPLIED ***
PAYMENT_DATE     YEAR_FRAC    RATE(%)       FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020          -          -            -   1.00000000            -            -
FRI 20 MAR 2020  0.5055556    0.00000         0.00   1.00000000         0.00         0.00
MON 21 SEP 2020  0.5138889    4.91869    252765.98   0.97534656    246534.43    246534.43
MON 22 MAR 2021  0.5055556    4.92370    248920.14   0.95165788    236886.81    483421.24
MON 20 SEP 2021  0.5055556    4.93134    249306.60   0.92850952    231483.55    714904.79
MON 21 MAR 2022  0.5055556    4.93134    249306.60   0.90592423    225852.89    940757.68
TUE 20 SEP 2022  0.5083333    4.93167    250693.45   0.88376873    221555.03   1162312.71
MON 20 MAR 2023

And here is the fixed leg valuation

In [22]:
swap.printFixedLegPV()

START DATE: WED 20 JUN 2018
MATURITY DATE: MON 22 SEP 2025
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020          -            -   1.00000000            -            -
MON 21 SEP 2020  1.0194444    509722.22   0.97534656    497155.81    497155.81
MON 20 SEP 2021  1.0111111    505555.56   0.92850952    469413.15    966568.96
TUE 20 SEP 2022  1.0138889    506944.44   0.88376873    448021.65   1414590.61
WED 20 SEP 2023  1.0138889    506944.44   0.84118380    426433.45   1841024.06
FRI 20 SEP 2024  1.0166667    508333.33   0.80062036    406982.02   2248006.08
MON 22 SEP 2025  1.0194444    509722.22   0.76186481    388339.43   2636345.51


In [23]:
swap.pv01(valuationDate, discountCurve)

5.272691014585207

In [24]:
swap.parCoupon(valuationDate, discountCurve)

0.04516387811692542

## Including a final exchange of principal

In [25]:
swap.value(settlementDate, discountCurve, discountCurve, principal=1.0)

254993.64637966827

In [27]:
swap.printFloatLegPV()

START DATE: WED 20 JUN 2018
MATURITY DATE: MON 22 SEP 2025
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
         *** FIRST FLOATING RATE PAYMENT IS IMPLIED ***
PAYMENT_DATE     YEAR_FRAC    RATE(%)       FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020          -          -            -   1.00000000            -            -
FRI 20 MAR 2020  0.5055556    0.00000         0.00   1.00000000         0.00         0.00
MON 21 SEP 2020  0.5138889    4.91869    252765.98   0.97534656    246534.43    246534.43
MON 22 MAR 2021  0.5055556    4.92370    248920.14   0.95165788    236886.81    483421.24
MON 20 SEP 2021  0.5055556    4.93134    249306.60   0.92850952    231483.55    714904.79
MON 21 MAR 2022  0.5055556    4.93134    249306.60   0.90592423    225852.89    940757.68
TUE 20 SEP 2022  0.5083333    4.93167    250693.45   0.88376873    221555.03   1162312.71
MON 20 MAR 2023

As expected we see the final PV of the floating Leg is par (same index and discounting curve).

In [29]:
swap.printFixedLegPV()

START DATE: WED 20 JUN 2018
MATURITY DATE: MON 22 SEP 2025
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020          -            -   1.00000000            -            -
MON 21 SEP 2020  1.0194444    509722.22   0.97534656    497155.81    497155.81
MON 20 SEP 2021  1.0111111    505555.56   0.92850952    469413.15    966568.96
TUE 20 SEP 2022  1.0138889    506944.44   0.88376873    448021.65   1414590.61
WED 20 SEP 2023  1.0138889    506944.44   0.84118380    426433.45   1841024.06
FRI 20 SEP 2024  1.0166667    508333.33   0.80062036    406982.02   2248006.08
MON 22 SEP 2025  1.0194444  10509722.22   0.76186481   8006987.57  10254993.65


Copyright (c) 2020 Dominic O'Kane